## Import the first Pretrained model:
- The model must be trained on the specific datasets redundant and not
- test the model by using Matrix Entropy
- Report results and use different models

In [1]:
import pandas as pd

direct_df = pd.read_csv('direct_data.csv')
redundant_df = pd.read_csv('redundant_data.csv')


In [2]:
from transformers import AutoTokenizer

model_name = "bert-base-uncased"  # Choose a model appropriate for your task
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the data
def tokenize_data(df):
    return tokenizer(df['input'].tolist(), padding="max_length", truncation=True, max_length=512)

#direct_encoded = tokenize_data(direct_df)
#redundant_encoded = tokenize_data(redundant_df)


In [3]:
from sklearn.model_selection import train_test_split

# Assuming 'data_df' is one of your DataFrames from direct_data or redundant_data
train_df, eval_df = train_test_split(direct_dataset, test_size=0.1)  # 10% for evaluation


NameError: name 'direct_dataset' is not defined

In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = CustomDataset(tokenize_data(train_df))
eval_dataset = CustomDataset(tokenize_data(eval_df))



In [5]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Now the trainer also knows about the eval dataset
)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
trainer.train()
evaluation_results = trainer.evaluate()
print(evaluation_results)

